In [7]:
import os
import re
import pandas as pd
import text_input
import importlib

## Parameters

In [2]:
report  = "R3"
path2SP = f"/Users/{os.getlogin()}/OneDrive - World Justice Project/EU Subnational/EU-S Data/reports/eu-thematic-reports"

## Reading outline

In [130]:
# Reading outline
outline = pd.read_excel(f"{path2SP}/data-viz/inputs/report_outline.xlsx")
charts4reports = (
    outline.copy()
    .loc[(outline["thematic_reports"] == True)]
)

# Extracting the section from column "Figure"
def get_section(fig):
    match = re.search(r"(?<=Figure )\d+\.\d+", fig)
    if match:
        section = match.group().strip()
        section = re.sub(r"\.", "_", section)
        return section
    else:
        return None
charts4reports["section"] = charts4reports["figure"].apply(get_section)

# Creating text input classes

In [164]:
# Reading text inputs
input_files = [
    "title-&-scroll.md",
    "introduction.md",
    "executive-summary.md",
    "thematic-findings.md",
    "methodology.md",
    "about.md",
    "acknowledgements.md"
]
inputs_as_classes = [text_input.text_input(report, x, path2SP) for x in input_files]

## Wrangling and exporting data

In [174]:
input_data = [x.get_csv(charts4reports) for x in inputs_as_classes]
front_page = [inputs_as_classes[0].get_front_page()]
csv_data   = pd.concat(front_page + input_data)
csv_data.to_csv(f"{path2SP}/html/{report}/{report}-csv-schema.csv", index = False, encoding = "utf-8") 